In [1]:
import json
import networkx as nx
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
def create_graph(filename):
    datafile = open(filename)
    tweets = json.load(datafile)
    graph = nx.Graph()

    for key, value in tweets.items():
        for friend in value["friends"]:
            if key != friend:
                graph.add_edge(key, friend)
    return graph

In [3]:
def jaccard_score(neighbor_nodes, neighbor_nodes2):
    score = len(neighbor_nodes & neighbor_nodes2) / len(neighbor_nodes | neighbor_nodes2)
    return score

In [4]:
def draw_network(graph, filename, min_degree=5):
    labels = {}
    for node in graph.nodes():
        if graph.degree(node) > min_degree:
            labels[node] = node
        else:
            labels[node] = ''
    plt.figure()
    plt.axis('off')
    nx.draw_networkx(graph, alpha=0.95, width=0.7, edge_color='#C0C0C0', labels=labels, node_size=30, font_size=10)
    plt.savefig(filename, dpi=200)

In [5]:
def find_best_edge(graph):
    eb = nx.edge_betweenness_centrality(graph)
    sorted_eb = sorted(eb.items(), key=lambda x: x[1], reverse=True)
    return sorted_eb[0][0]

In [6]:
def girvan_newman(graph):
    if len(graph.nodes()) == 1:
        return [graph.nodes()]

    components = list(nx.connected_component_subgraphs(graph))
    while len(components) == 1:
        edge = find_best_edge(graph)
        graph.remove_edge(edge[0], edge[1])
        components = list(nx.connected_component_subgraphs(graph))
    return components

In [7]:
def detect_communities(graph):
    components = girvan_newman(graph)
    communities = dict()

    for idx, g in enumerate(components):
        communities[idx + 1] = g.nodes()
        # print("Component %d - %d nodes" % (idx + 1, len(g.nodes())))
        # draw_network(g, "components/component-%d.png" % (idx + 1), min_degree=1)
    return communities

In [8]:
def save_results(communities, filename):
    with open(filename, "w") as outfile:
        json.dump(communities, outfile)


def compress_graph(graph, threshold=0, min_degree=1):
    # Remove edges whose Jaccard Score is 0
    for u1, u2 in graph.edges():
        u1_neighbors = set(graph.neighbors(u1))
        u2_neighbors = set(graph.neighbors(u2))
        score = jaccard_score(u1_neighbors, u2_neighbors)

        if not score > threshold:
            graph.remove_edge(u1, u2)

    # Remove nodes who don't have any edges
    for node in graph.nodes():
        if graph.degree(node) < min_degree:
            graph.remove_node(node)

In [10]:
if __name__ == '__main__':
    graph = create_graph("tweets.json")
    # print('graph has %d nodes and %d edges' %
    #       (graph.order(), graph.number_of_edges()))

    compress_graph(graph)
    # print('compressed graph has %d nodes and %d edges' %
    #       (graph.order(), graph.number_of_edges()))

    draw_network(graph, "graph.png")

    girvan_newman(graph)
    communities = detect_communities(graph)
    save_results(communities, "communities.json")

In [ ]:
`